# 📝 Exercise M4.02

In the previous notebook, we showed that we can add new features based on the
original feature `x` to make the model more expressive, for instance `x ** 2` or
`x ** 3`. In that case we only used a single feature in `data`.

The aim of this notebook is to train a linear regression algorithm on a
dataset with more than a single feature. In such a "multi-dimensional" feature
space we can derive new features of the form `x1 * x2`, `x2 * x3`, etc.
Products of features are usually called "non-linear" or "multiplicative"
interactions between features.

Feature engineering can be an important step of a model pipeline as long as
the new features are expected to be predictive. For instance, think of a
classification model to decide if a patient has risk of developing a heart
disease. This would depend on the patient's Body Mass Index which is defined
as `weight / height ** 2`.

We load the dataset penguins dataset. We first use a set of 3 numerical
features to predict the target, i.e. the body mass of the penguin.

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">If you want a deeper overview regarding this dataset, you can refer to the
Appendix - Datasets description section at the end of this MOOC.</p>
</div>

In [1]:
import pandas as pd

penguins = pd.read_csv("../datasets/penguins.csv")

columns = ["Flipper Length (mm)", "Culmen Length (mm)", "Culmen Depth (mm)"]
target_name = "Body Mass (g)"

# Remove lines with missing values for the columns of interest
penguins_non_missing = penguins[columns + [target_name]].dropna()

data = penguins_non_missing[columns]
target = penguins_non_missing[target_name]
data.head()

,Flipper Length (mm),Culmen Length (mm),Culmen Depth (mm)
0,181.0,39.1,18.7
1,186.0,39.5,17.4
2,195.0,40.3,18.0
4,193.0,36.7,19.3
5,190.0,39.3,20.6


Now it is your turn to train a linear regression model on this dataset. First,
create a linear regression model.

In [2]:
# def fit_score_plot_regression(model, title=None):
#     model.fit(data, target)
#     target_predicted = model.predict(data)
#     mse = mean_squared_error(target, target_predicted)
#     ax = sns.scatterplot(
#         data=full_data, x="input_feature", y="target", color="black", alpha=0.5
#     )
#     ax.plot(data, target_predicted)
#     if title is not None:
#         _ = ax.set_title(title + f" (MSE = {mse:.2f})")
#     else:
#         _ = ax.set_title(f"Mean squared error = {mse:.2f}")
        
        
from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error

linear_regression = LinearRegression()
# linear_regression.fit(data, target)


LinearRegression()

Execute a cross-validation with 10 folds and use the mean absolute error (MAE)
as metric.

In [5]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(linear_regression, data, target, cv=10, scoring="neg_mean_absolute_error") #bast mae = 0
cv_results

{'fit_time': array([0.00123811, 0.00199819, 0.00157666, 0.00200105, 0.00199723,
        0.00099897, 0.0016253 , 0.00099969, 0.00100231, 0.00100112]),
 'score_time': array([0.0112915 , 0.        , 0.00093913, 0.00100183, 0.00200295,
        0.0015161 , 0.00088906, 0.00099921, 0.0009973 , 0.0010097 ]),
 'test_score': array([-423.71330276, -324.58692583, -300.28904542, -370.38769817,
        -358.78336272, -288.95569995, -205.46078515, -254.84373443,
        -320.21473882, -523.4780806 ])}

Compute the mean and std of the MAE in grams (g). Remember you have to revert
the sign introduced when metrics start with `neg_`, such as in
`"neg_mean_absolute_error"`.

In [8]:
print(f'mean : {-cv_results["test_score"].mean()}')
print(f'std : {cv_results["test_score"].std()}')


mean : 337.0713373844392
std : 84.86840942516258


Now create a pipeline using `make_pipeline` consisting of a
`PolynomialFeatures` and a linear regression. Set `degree=2` and
`interaction_only=True` to the feature engineering step. Remember not to
include a "bias" feature (that is a constant-valued feature) to avoid
introducing a redundancy with the intercept of the subsequent linear
regression model.

You may want to use the `.set_output(transform="pandas")` method of the
pipeline to answer the next question.

In [14]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

polynomial_regression = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False, interaction_only=True),
    LinearRegression(),
).set_output(transform="pandas") #to convert the output in dataframe 



Transform the first 5 rows of the dataset and look at the column names. How
many features are generated at the output of the `PolynomialFeatures` step in
the previous pipeline?

In [16]:
polynomial_regression.fit(data, target)
polynomial_regression[0].transform(data[0:5])

,Flipper Length (mm),Culmen Length (mm),Culmen Depth (mm),Flipper Length (mm) Culmen Length (mm),Flipper Length (mm) Culmen Depth (mm),Culmen Length (mm) Culmen Depth (mm)
0,181.0,39.1,18.7,7077.1,3384.7,731.17
1,186.0,39.5,17.4,7347.0,3236.4,687.30
2,195.0,40.3,18.0,7858.5,3510.0,725.40
4,193.0,36.7,19.3,7083.1,3724.9,708.31
5,190.0,39.3,20.6,7467.0,3914.0,809.58


Check that the values for the new interaction features are correct for a few
of them.

In [17]:
flipper_length = 181
culmen_depth = 18
flipper_length * culmen_depth

3258

Use the same cross-validation strategy as done previously to estimate the mean
and std of the MAE in grams (g) for such a pipeline. Compare with the results
without feature engineering.

In [18]:
cv_results = cross_validate(polynomial_regression, data, target, cv=10, scoring="neg_mean_absolute_error") #bast mae = 0

print(f'mean : {-cv_results["test_score"].mean()}')
print(f'std : {cv_results["test_score"].std()}')


mean : 301.78955228432073
std : 44.34000781940749



Now let's try to build an alternative pipeline with an adjustable number of
intermediate features while keeping a similar predictive power. To do so, try
using the `Nystroem` transformer instead of `PolynomialFeatures`. Set the
kernel parameter to `"poly"` and `degree` to 2. Adjust the number of
components to be as small as possible while keeping a good cross-validation
performance.

Hint: Use a `ValidationCurveDisplay` with `param_range = np.array([5, 10, 50,
100])` to find the optimal `n_components`.

In [24]:
from sklearn.kernel_approximation import Nystroem
from sklearn.model_selection import ValidationCurveDisplay
import numpy as np

nystroem_regression = make_pipeline(
    Nystroem(kernel="poly", degree=2, n_components=10, random_state=0), #best choice 10 from validation curve
    LinearRegression(),
)
# nystroem_regression.get_params()

# param_range = np.array([5, 10, 50, 100])

# disp = ValidationCurveDisplay.from_estimator(
#     nystroem_regression,
#     data,
#     target,
#     param_name="nystroem__n_components",
#     param_range=param_range,
#     cv=10,
#     scoring="neg_mean_absolute_error",
#     n_jobs=2,
# )
# _ = disp.ax_.set(
#     xlabel="Number of Components",
#     ylabel="Mae",
#     title="Validation curve",
# )



cv_results = cross_validate(nystroem_regression, data, target, cv=10, scoring="neg_mean_absolute_error") #bast mae = 0




How do the mean and std of the MAE for the Nystroem pipeline with optimal
`n_components` compare to the other previous models?

In [25]:
print(f'mean : {-cv_results["test_score"].mean()}')
print(f'std : {cv_results["test_score"].std()}')


mean : 299.2828167006963
std : 44.99023342961616
